In [1]:
import pandas as pd
import numpy as np

##### Data exploration

In [2]:
d = pd.read_csv('waste_cluster.csv')
d

,cluster,demand
0,C1,3
1,C2,4
2,C3,3
3,C4,4
4,C5,3
5,C6,4
6,C7,3


In [3]:
cs_distance = pd.read_excel('distance_clusters_storages.xlsx', 
                   header=[0,1], 
                   index_col=[0,1])

cs_distance

storage                           
                 S1   S2    S3   S4   S5    S6
distance C1     3.8  5.7  16.8  1.5  5.7  13.0
         C2     4.4  5.0  16.6  9.2  6.0  12.8
         C3     6.2  6.9  19.5  2.5  3.4  10.3
         C4     5.2  4.5  17.0  0.3  6.0  12.5
         C5     2.6  7.0  17.0  3.0  5.7  13.9
         C6     2.0  7.5  16.7  3.7  6.2  14.5
         C7     3.5  6.2  17.0  7.0  5.4  13.2

In [4]:
cs_distance.columns = cs_distance.columns.droplevel()
cs_distance.index = cs_distance.index.droplevel()
cs_distance

,S1,S2,S3,S4,S5,S6
C1,3.8,5.7,16.8,1.5,5.7,13.0
C2,4.4,5.0,16.6,9.2,6.0,12.8
C3,6.2,6.9,19.5,2.5,3.4,10.3
C4,5.2,4.5,17.0,0.3,6.0,12.5
C5,2.6,7.0,17.0,3.0,5.7,13.9
C6,2.0,7.5,16.7,3.7,6.2,14.5
C7,3.5,6.2,17.0,7.0,5.4,13.2


In [5]:
fs_distance = pd.read_excel('distance_facilities_storages.xlsx', 
                   header=[0,1], 
                   index_col=[0,1])

fs_distance

facility             storage                              
                  F1    F2    F3      S1    S2    S3    S4    S5    S6
facility F1      inf   inf   inf    13.0   7.5  21.0   7.0   8.5   8.5
         F2      inf   inf   inf    23.0  22.0  35.0  18.6  14.5   6.0
         F3      inf   inf   inf     4.0  12.0  15.0   8.5  11.0  17.5
storage  S1     13.0  23.0   4.0     inf   8.0  19.0   5.0   9.5  16.5
         S2      7.5  22.0  12.0     8.0   inf  17.0   4.5   9.0  14.5
         S3     21.0  35.0  15.0    19.0  17.0   inf  21.0  28.0  33.5
         S4      7.0  18.6   8.5     5.0   4.5  21.0   inf   5.0  15.0
         S5      8.5  14.5  11.0     9.5   9.0  28.0   5.0   inf  10.0
         S6      8.5   6.0  17.5    16.5  14.5  33.5  15.0  10.0   inf

In [6]:
df_storage_info = pd.read_csv('cost_capacity_storages.csv')
df_storage_info

,storage,cost,capacity
0,S1,100,8
1,S2,100,8
2,S3,100,8
3,S4,100,6
4,S5,100,6
5,S6,100,6


### Model Formulation

In [7]:
import gurobipy as gp
from gurobipy import GRB

#### Setting Parameters

In [8]:
storages = list(df_storage_info.storage)
m_storages = len(storages)

fields = cs_distance.index.tolist()
n_fields = len(fields)

k_vehicles = 3
Q_vehicle_capacity = 10
facility = 'F3'

J_0 = storages + [facility]

In [9]:
series_fs_distance = fs_distance.loc[('facility',facility), 'storage']
series_fs_distance.name = facility
df_fs_distance = series_fs_distance.to_frame()
df_fs_distance

,F3
S1,4.0
S2,12.0
S3,15.0
S4,8.5
S5,11.0
S6,17.5


In [10]:
missing_row = pd.concat([series_fs_distance,pd.Series([np.inf], index=[facility])])
missing_row = missing_row.to_frame()
missing_row.columns = [facility]
missing_row = missing_row.T
missing_row

,S1,S2,S3,S4,S5,S6,F3
F3,4.0,12.0,15.0,8.5,11.0,17.5,inf


In [11]:
fs_distance = pd.merge(fs_distance.loc[('storage',), 'storage'], df_fs_distance, left_index=True, right_index=True)
fs_distance = pd.concat([fs_distance, missing_row])
fs_distance

,S1,S2,S3,S4,S5,S6,F3
S1,inf,8.0,19.0,5.0,9.5,16.5,4.0
S2,8.0,inf,17.0,4.5,9.0,14.5,12.0
S3,19.0,17.0,inf,21.0,28.0,33.5,15.0
S4,5.0,4.5,21.0,inf,5.0,15.0,8.5
S5,9.5,9.0,28.0,5.0,inf,10.0,11.0
S6,16.5,14.5,33.5,15.0,10.0,inf,17.5
F3,4.0,12.0,15.0,8.5,11.0,17.5,inf


In [12]:
fields_idx = dict(zip(fields, range(n_fields)))
storages_idx = dict(zip(storages, range(m_storages)))
idx_to_storages = dict(zip(range(m_storages), storages))
J_0_idx = dict(zip(J_0, range(len(J_0))))

print('fields idx:', fields_idx)
print('storages idx:', storages_idx)
print('idx to storages:', idx_to_storages)
print('J_0 idx:', J_0_idx)

fields idx: {'C1': 0, 'C2': 1, 'C3': 2, 'C4': 3, 'C5': 4, 'C6': 5, 'C7': 6}
storages idx: {'S1': 0, 'S2': 1, 'S3': 2, 'S4': 3, 'S5': 4, 'S6': 5}
idx to storages: {0: 'S1', 1: 'S2', 2: 'S3', 3: 'S4', 4: 'S5', 5: 'S6'}
J_0 idx: {'S1': 0, 'S2': 1, 'S3': 2, 'S4': 3, 'S5': 4, 'S6': 5, 'F3': 6}


In [13]:
f = dict(zip(df_storage_info.storage, df_storage_info.cost))
q = dict(zip(df_storage_info.storage, df_storage_info.capacity))

print('costs to open a facility:', f)
print('capacity of facility:', q)

costs to open a facility: {'S1': 100, 'S2': 100, 'S3': 100, 'S4': 100, 'S5': 100, 'S6': 100}
capacity of facility: {'S1': 8, 'S2': 8, 'S3': 8, 'S4': 6, 'S5': 6, 'S6': 6}


In [14]:
d = dict(zip(d.cluster, d.demand))
print('amount of agricaltural waste:', d)

amount of agricaltural waste: {'C1': 3, 'C2': 4, 'C3': 3, 'C4': 4, 'C5': 3, 'C6': 4, 'C7': 3}


#### Decision Variables

In [15]:
larp_model = gp.Model() # general Gurobi mdodel
larp_model = gp.Model() # general Gurobi mdodel
larp_model.modelSense = GRB.MAXIMIZE # decleare the problem as minimization problem

X = larp_model.addVars([j for j in range(m_storages)], vtype=GRB.BINARY)
Y = larp_model.addVars([(i,j) for i in range(n_fields) for j in range(m_storages)], vtype=GRB.BINARY)
Z = larp_model.addVars([(u,v) for u in range(len(J_0)) for v in range(len(J_0)) if u!=v], vtype=GRB.BINARY)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-12


##### Auxiliary Decision Variables

In [16]:
T = larp_model.addVars([w for w in range(m_storages)], vtype=GRB.INTEGER)

for w in storages:
    larp_model.addConstr(T[storages_idx[w]] >= 0)

#### Objective Function

In [17]:
larp_model.setObjective(
    gp.quicksum(f[j]*X[storages_idx[j]] for j in storages) +
    gp.quicksum(cs_distance.loc[i,j]*d[i]*Y[fields_idx[i],storages_idx[j]] for i in fields for j in storages) +
    gp.quicksum(fs_distance.loc[u,v]*Z[J_0_idx[u],J_0_idx[v]] for u in J_0 for v in J_0 if u!=v)
)

#### Model Constrains

In [18]:
for i in fields:
    larp_model.addConstr(gp.quicksum(Y[fields_idx[i],storages_idx[j]] for j in storages) == 1)

In [19]:
for j in storages:
    larp_model.addConstr(gp.quicksum(d[i]*Y[fields_idx[i],storages_idx[j]] for i in fields) <= q[j]*X[storages_idx[j]])

In [20]:
larp_model.addConstr(gp.quicksum(Z[J_0_idx[u],J_0_idx[facility]] for u in storages) == k_vehicles)
larp_model.addConstr(gp.quicksum(Z[J_0_idx[facility],J_0_idx[v]] for v in storages) == k_vehicles)

<gurobi.Constr *Awaiting Model Update*>

-----------------------------------------------------------------------------------------------------------------------

#### Linearization of non-linear constrains

##### Additional Decision Variables

In [21]:
W_1 = larp_model.addVars([(u,v) for u in range(len(J_0)) for v in range(len(J_0)) if u!=v], vtype=GRB.BINARY)
W_2 = larp_model.addVars([(u,v) for u in range(len(J_0)) for v in range(len(J_0)) if u!=v], vtype=GRB.BINARY)

##### Additional Constrains

In [25]:
for v in storages:
    larp_model.addConstr(gp.quicksum(W_1[J_0_idx[u], storages_idx[v]] for u in J_0 if u!=v) == X[storages_idx[v]])

for u in J_0:
    for v in J_0:
        if u!=v and u in storages:
            larp_model.addConstr(W_1[J_0_idx[u], J_0_idx[v]] <= X[storages_idx[u]])
            larp_model.addConstr(
                W_1[J_0_idx[u], J_0_idx[v]] >= X[storages_idx[u]] + Z[J_0_idx[u], J_0_idx[v]] - 1
            )
        if u!=v:
            larp_model.addConstr(W_1[J_0_idx[u], J_0_idx[v]] <= Z[J_0_idx[u], J_0_idx[v]])


In [26]:
for u in storages:
    larp_model.addConstr(gp.quicksum(W_2[storages_idx[u], J_0_idx[v]] for v in J_0 if u!=v) == X[storages_idx[u]])

for u in J_0:
    for v in J_0:
        if u!=v and v in storages:
            larp_model.addConstr(W_2[J_0_idx[u], J_0_idx[v]] <= X[storages_idx[v]])
            larp_model.addConstr(
                W_2[J_0_idx[u], J_0_idx[v]] >= X[storages_idx[v]] + Z[J_0_idx[u], J_0_idx[v]] - 1
            )
        if u!=v:
            larp_model.addConstr(W_2[J_0_idx[u], J_0_idx[v]] <= Z[J_0_idx[u], J_0_idx[v]])

-----------------------------------------------------------------------------------------------------------------------

In [27]:
for u in storages:
    for v in storages:
        if u != v:
            larp_model.addConstr(T[storages_idx[u]] - T[storages_idx[v]] + Q_vehicle_capacity*Z[storages_idx[u], storages_idx[v]] <= 
                                 Q_vehicle_capacity - k_vehicles**(-1)*gp.quicksum(d[i]*Y[fields_idx[i],storages_idx[v]] for i in fields))

    larp_model.addConstr(k_vehicles**(-1)*gp.quicksum(d[i]*Y[fields_idx[i],storages_idx[u]] for i in fields) <= T[storages_idx[u]])
    larp_model.addConstr(T[storages_idx[u]] <= Q_vehicle_capacity)

In [28]:
for i in fields:
    for j in storages:
        larp_model.addConstr(Y[fields_idx[i],storages_idx[j]] >= 0)

In [29]:
for u in storages:
    larp_model.addConstr(T[storages_idx[u]] >= 0)

#### Optimization

In [30]:
larp_model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-5200U CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 351 rows, 180 columns and 1122 nonzeros
Model fingerprint: 0x644537b0
Variable types: 0 continuous, 180 integer (174 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Found heuristic solution: objective 643.4000000
Presolve removed 186 rows and 42 columns
Presolve time: 0.05s
Presolved: 165 rows, 138 columns, 798 nonzeros
Variable types: 0 continuous, 138 integer (132 binary)
Found heuristic solution: objective 654.9000000

Root relaxation: objective 1.082883e+03, 88 iterations, 0.02 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth

#### Solutions

In [31]:
X_sol = np.array([X[storages_idx[j]].x for j in storages])
print(X_sol)

X_sol_rapresentation = [idx_to_storages[x] for x in range(len(X_sol)) if X_sol[x] > 0.5]
print(X_sol_rapresentation)

[1. 1. 1. 1. 1. 1.]
['S1', 'S2', 'S3', 'S4', 'S5', 'S6']


In [32]:
Y_sol = np.array([[Y[fields_idx[i],storages_idx[j]].x for j in storages] for i in fields])
Y_sol_rapresentation = pd.DataFrame(Y_sol, columns=storages, index=fields)
Y_sol_rapresentation

,S1,S2,S3,S4,S5,S6
C1,-0.0,0.0,-0.0,0.0,-0.0,1.0
C2,-0.0,-0.0,-0.0,1.0,-0.0,0.0
C3,-0.0,-0.0,1.0,0.0,-0.0,0.0
C4,-0.0,0.0,1.0,0.0,-0.0,0.0
C5,0.0,1.0,0.0,0.0,0.0,0.0
C6,0.0,1.0,-0.0,0.0,-0.0,0.0
C7,0.0,0.0,0.0,-0.0,0.0,1.0


In [33]:
Z_sol = np.array([[Z[J_0_idx[u],J_0_idx[v]].x if u!=v else 0.0 for v in J_0] for u in J_0])
Z_sol_rapresentation = pd.DataFrame(Z_sol, columns=fs_distance.columns, index=fs_distance.index)
Z_sol_rapresentation

,S1,S2,S3,S4,S5,S6,F3
S1,0.0,0.0,0.0,-0.0,1.0,0.0,-0.0
S2,1.0,0.0,0.0,-0.0,-0.0,0.0,-0.0
S3,0.0,0.0,0.0,-0.0,-0.0,1.0,1.0
S4,0.0,-0.0,0.0,0.0,0.0,-0.0,1.0
S5,-0.0,0.0,1.0,0.0,0.0,0.0,-0.0
S6,0.0,0.0,0.0,1.0,0.0,0.0,1.0
F3,-0.0,1.0,1.0,-0.0,-0.0,1.0,0.0


In [34]:
T_sol = np.array([T[w].x for w in range(m_storages)])
print(T_sol)

T_sol_rapresentation = [idx_to_storages[t] for t in range(len(T_sol))]
print(T_sol_rapresentation)

[ 3.  3.  6. 10.  3.  8.]
['S1', 'S2', 'S3', 'S4', 'S5', 'S6']


In [38]:
W_1_sol = np.array([[W_1[u,v].x if u!=v else np.inf for v in range(len(J_0))] for u in range(len(J_0))])
W_1_sol_rapresentation = pd.DataFrame(W_1_sol, columns=fs_distance.columns, index=fs_distance.index)
W_1_sol_rapresentation

,S1,S2,S3,S4,S5,S6,F3
S1,inf,0.0,-0.0,-0.0,1.0,-0.0,0.0
S2,1.0,inf,-0.0,-0.0,-0.0,0.0,0.0
S3,0.0,0.0,inf,-0.0,0.0,1.0,1.0
S4,0.0,-0.0,-0.0,inf,-0.0,-0.0,1.0
S5,-0.0,-0.0,1.0,-0.0,inf,-0.0,0.0
S6,-0.0,-0.0,0.0,1.0,-0.0,inf,1.0
F3,0.0,1.0,0.0,-0.0,-0.0,0.0,inf


In [39]:
W_2_sol = np.array([[W_2[u,v].x if u!=v else np.inf for v in range(len(J_0))] for u in range(len(J_0))])
W_2_sol_rapresentation = pd.DataFrame(W_2_sol, columns=fs_distance.columns, index=fs_distance.index)
W_2_sol_rapresentation

,S1,S2,S3,S4,S5,S6,F3
S1,inf,-0.0,0.0,0.0,1.0,0.0,0.0
S2,1.0,inf,0.0,0.0,0.0,-0.0,-0.0
S3,-0.0,-0.0,inf,0.0,-0.0,1.0,0.0
S4,-0.0,0.0,0.0,inf,0.0,0.0,1.0
S5,0.0,0.0,1.0,0.0,inf,0.0,-0.0
S6,0.0,0.0,-0.0,1.0,0.0,inf,0.0
F3,0.0,1.0,1.0,0.0,0.0,1.0,inf
